In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd


In [2]:
ds = xr.open_dataset("ZA_chirps_monthly.nc")
ds_tmax = xr.open_dataset("ZA_chirts_monthly_tmax.nc")
ds_tmin = xr.open_dataset("ZA_chirts_monthly_tmin.nc")
# SVM alghorithm  for missing data(NAN) on ds
df=ds['precip'].to_dataframe().reset_index()
len_nan=df['precip'].isna().sum().sum()
df=df.sample(n=10000,random_state=42)
df['time']=(pd.to_datetime(df['time'])-pd.Timestamp('1981-01-01')).dt.days
# SVM alghorithm  for missing data(NAN) on tmax
df_tmax=ds_tmax['temp'].to_dataframe().reset_index()
len_nan_tmax=df_tmax['temp'].isna().sum().sum()
df_tmax=df_tmax.sample(n=10000,random_state=42)
df_tmax['time']=(pd.to_datetime(df_tmax['time'])-pd.Timestamp('1981-01-01')).dt.days
print(df_tmax.isna().sum().sum())
# SVM alghorithm  for missing data(NAN) on tmin
df_tmin=ds_tmin['temp'].to_dataframe().reset_index()
len_nan_tmin=df_tmin['temp'].isna().sum().sum()
df_tmin=df_tmin.sample(n=10000,random_state=42)
df_tmin['time']=(pd.to_datetime(df_tmin['time'])-pd.Timestamp('1981-01-01')).dt.days
print(df_tmin.isna().sum().sum())


2870
2870


In [3]:
#SVM alghorithm df
df_notNa=df[df['precip'].notna()]
df_na=df[df['precip'].isna()]
X=df_notNa.drop(columns='precip')
Y=df_notNa['precip']
#SVM alghorithm for tmax
df_tmax_notNa=df_tmax[df_tmax['temp'].notna()]
df_tmax_na=df_tmax[df_tmax['temp'].isna()]
X_tmax=df_tmax_notNa.drop(columns='temp')
Y_tmax=df_tmax_notNa['temp']
#SVM alghorithm for tmin
df_tmin_notNa=df_tmin[df_tmin['temp'].notna()]
df_tmin_na=df_tmin[df_tmin['temp'].isna()]
X_tmin=df_tmin_notNa.drop(columns='temp')
Y_tmin=df_tmin_notNa['temp']


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
#svm FIT for ds
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
svm=SVR(kernel='rbf')
svm.fit(x_train,y_train)
#svm FIT for tmax
x_train,x_test,y_train,y_test=train_test_split(X_tmax,Y_tmax,test_size=0.2,random_state=42)
svm_tmax=SVR(kernel='rbf')
svm_tmax.fit(x_train,y_train)
#svm FIT for tmin
x_train,x_test,y_train,y_test=train_test_split(X_tmin,Y_tmin,test_size=0.2,random_state=42)
svm_tmin=SVR(kernel='rbf')
svm_tmin.fit(x_train,y_train)



SVR()

In [5]:
#prediction of missing data on dataset of ds
X_missing=df_na.drop(columns='precip')
y_pred=svm.predict(X_missing)
y_pred
#prediction missing data on dataset of tmax
X_missing_tmax=df_tmax_na.drop(columns='temp')
y_pred_tmax=svm.predict(X_missing_tmax)
y_pred_tmax
#prediction missing data on dataset of tmin
X_missing_tmin=df_tmin_na.drop(columns='temp')
y_pred_tmin=svm.predict(X_missing_tmin)
y_pred_tmin


array([14.06533639, 15.12111154, 15.65637976, ..., 15.66857945,
       16.36781222, 16.53249463])

In [6]:
#filling missing data on df1
df1=ds['precip'].to_dataframe().reset_index()
y_pred=np.random.choice(y_pred,size=len_nan,replace=True)
df1.loc[df1['precip'].isna(),'precip']=y_pred
#filling missing data on tmax1
df_tmax1=ds_tmax['temp'].to_dataframe().reset_index()
y_pred_tmax=np.random.choice(y_pred_tmax,size=len_nan_tmax,replace=True)
df_tmax1.loc[df_tmax1['temp'].isna(),'temp']=y_pred_tmax
#filling missing data on tmin1
df_tmin1=ds_tmin['temp'].to_dataframe().reset_index()
y_pred_tmin=np.random.choice(y_pred_tmin,size=len_nan_tmin,replace=True)
df_tmin1.loc[df_tmin1['temp'].isna(),'temp']=y_pred_tmin
print(df_tmin1.isna().sum().sum())

C:\Users\USER\AppData\Local\Temp\ipykernel_1040\261819596.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[14.7462743  16.45002384 14.23269341 ... 13.99731839 15.48690517
 15.96761895]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df1.loc[df1['precip'].isna(),'precip']=y_pred
C:\Users\USER\AppData\Local\Temp\ipykernel_1040\261819596.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[16.58906675 15.93270618 14.07149822 ... 14.52326876 16.52363689
 16.35832554]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df_tmax1.loc[df_tmax1['temp'].isna(),'temp']=y_pred_tmax
C:\Users\USER\AppData\Local\Temp\ipykernel_1040\261819596.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future v

0


In [7]:
ds1=df1.set_index(['time','latitude','longitude']).to_xarray()
ds_tmax1=df_tmax1.set_index(['time','latitude','longitude']).to_xarray()
ds_tmin1=df_tmin1.set_index(['time','latitude','longitude']).to_xarray()


In [9]:
import rioxarray
crop_mask=rioxarray.open_rasterio('ZA_cropmask.tif')
crop_mask=crop_mask.squeeze()
print(crop_mask)
precip_masked=ds1.where(crop_mask !=0)
ds_tmax1_masked=ds_tmax1.where(crop_mask !=0)
ds_tmin1_masked=ds_tmin1.where(crop_mask !=0)